# Population Dynamics

## Algorithm

### 1. Obtain an equilibrated population for the cavity precisions

1. Start a random complex initial population $\hat{P} = (\omega_{1}, \dots, \omega_{N_{p}})$.
2. Performing at most `max_iter` iterations, for each iteration do:
    1. Pick 

1. Set: 
    1. Population size $N_{p}$, 
    1. Mean connectivity $c$, 
    1. Disorder $W$,
    1. Epsilon $\epsilon$, 
    1. Eigenvalue $\lambda$,
    1. Maximum number of iterations.
    1. Maximum absolute change.

1. Create an initial complex population of cavity precisions $\hat{P} = (\omega_{1}, \dots, \omega_{N_{p}})$.

1. Pick $c - 1$ random elements $\left\{ \omega_{\ell} \right\}$ from $\hat{P}$ and sample $E$ from $\rho(E)$:
    \begin{equation}
        \rho(E) = \frac{1}{W} \Theta\left(\frac{W}{2} - \lvert E \rvert \right)
    \end{equation}
    
1. Replace a random element of the population with
    \begin{equation}
        i(\lambda - i \epsilon - E) + \sum_{\ell = 1}^{c-1} \frac{1}{\omega_{\ell}}
    \end{equation}

1. Repeat the two previous steps until equilibrium is reached (changes are 'consistently' below tolerance)

In [1]:
using ProgressMeter
using Distributions
using Random

## Create an equilibrated population of cavity precisions

In [2]:
Random.seed!(42)

Np = 10^3  # Population size.
c = 3  # Mean connectivity.
W = 0.3
ϵ = 1e-3
λ = 0.

tolerance = 1e-6
max_iters = 100*Np

cavity_ω = rand(Complex{Float64}, Np);

counter = 0
ω_average = 0

final_Δω = 0

for iteration in 1:max_iters
    # Last index gives the cavity precision that will be changed.
    rand_ids = sample(1:Np, c; replace=false)
    chosen_ω = cavity_ω[rand_ids[1:end-1]]
    
    idx = rand_ids[end]
    
    E = rand(Uniform(-W/2., W/2.))
    
    old_ω = cavity_ω[idx]
    new_ω = im*(λ - im*ϵ - E) + sum(1 ./ chosen_ω)
    Δω = abs(new_ω - old_ω)
    
    cavity_ω[idx] = new_ω
    
    (Δω < tolerance) && println(iteration) && break
    
    counter += 1
    
    if counter == 100
        new_average = sum(cavity_ω)
        
        Δω = (new_average - ω_average) / Np
        
        Δω_real = abs(real(Δω))
        Δω_imag = abs(imag(Δω))
        
        max_Δω = max(Δω_real, Δω_imag)
        
#         println("Maximum change after $(iteration) iterations: $(max_Δω)")
        
        ω_average = new_average
        
        final_Δω = max_Δω
        
        counter = 0
    end
end

## Sample marginal precisions and measure spectral density

In [ ]:
Random.seed!(42)

N = 2^10  # Sample N values for the marginal precisions.

marginal_ω = []


function update_cavity_ω!(cavity_ω)
    # Last index gives the cavity precision that will be changed.
    rand_ids = sample(1:Np, c; replace=false)
    chosen_ω = cavity_ω[rand_ids[1:end-1]]
    
    idx = rand_ids[end]
    
    E = rand(Uniform(-W/2., W/2.))
    
    cavity_ω[idx] = im*(λ - im*ϵ - E) + sum(1 ./ chosen_ω)
    
    return 
end


for k in 1:N
    
    # Pick 'c' random elements from Peq...
    rand_ids = sample(1:Np, c; replace=false)
    chosen_ω = cavity_ω[rand_ids]
    
    # ...and sample E from ρ(E).
    E = rand(Uniform(-W/2., W/2.))
    
    # Generate a sample from Ω ∼ P(Ω)
    ω = im*(λ - im*ϵ - E) + sum(1 ./ chosen_ω)
    
    push!(marginal_ω, ω)
    
    # Update one random element from Peq.
    update_cavity_ω!(cavity_ω)
    
end

In [ ]:
sum(imag.(im ./ marginal_ω)) / (π * N)